In [95]:
import re

In [96]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [97]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [98]:
len(text)

205

In [99]:
num_characters = 34 #33 буквы + пробел

In [100]:
import keras
from keras.preprocessing.text import Tokenizer

In [101]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [102]:
tokenizer.fit_on_texts(text)

In [103]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [104]:
inp_chars = 3
data = tokenizer.texts_to_matrix(text)

In [105]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [106]:
n = data.shape[0]-inp_chars

In [107]:
import numpy as np

In [108]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [109]:
len(X)

202

In [110]:
X[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [111]:
Y[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [112]:
from keras.layers import *
from keras.models import Sequential

In [133]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(1024, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_13 (SimpleRNN)    (None, 1024)              1084416   
_________________________________________________________________
dense_13 (Dense)             (None, 34)                34850     
Total params: 1,119,266
Trainable params: 1,119,266
Non-trainable params: 0
_________________________________________________________________


In [134]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 0s 34ms/step - loss: 3.3945 - accuracy: 0.0743
Epoch 2/100
7/7 [==============================] - 0s 32ms/step - loss: 2.3960 - accuracy: 0.2970
Epoch 3/100
7/7 [==============================] - 0s 31ms/step - loss: 2.0137 - accuracy: 0.3762
Epoch 4/100
7/7 [==============================] - 0s 30ms/step - loss: 1.7476 - accuracy: 0.4356
Epoch 5/100
7/7 [==============================] - 0s 31ms/step - loss: 1.6143 - accuracy: 0.4455
Epoch 6/100
7/7 [==============================] - 0s 33ms/step - loss: 1.4429 - accuracy: 0.5198
Epoch 7/100
7/7 [==============================] - 0s 32ms/step - loss: 1.3299 - accuracy: 0.5347
Epoch 8/100
7/7 [==============================] - 0s 31ms/step - loss: 1.2782 - accuracy: 0.5594
Epoch 9/100
7/7 [==============================] - 0s 32ms/step - loss: 1.2184 - accuracy: 0.6139
Epoch 10/100
7/7 [==============================] - 0s 32ms/step - loss: 1.1856 - accuracy: 0.5594
Epoch 11/100
7/7 [=

In [135]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding
 
    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)
 
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении
 
    inp_str += d # дописываем строку
 
  return inp_str

In [136]:
res = buildPhrase("утренн")
print(res)

утреннрйе дте нсаво ио л вничыо иы л   учывуши льй ычио 


# Слова


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Пример текста для обучения
corpus = "Вы — лучший ответ на проблемы, которые возникли в понедельник. Думайте позитивно и верьте в свою способность достигать отличных результатов. Если вы смогли в понедельник подняться с постели, значит вы супер герой."

In [3]:
# Преобразование входных данных
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

In [4]:
# Создание признаков и меток
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

In [5]:
# Определение модели нейронной сети
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(predictors, label, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 0s 999us/step - loss: 3.3327 - accuracy: 0.0323
Epoch 2/100
1/1 [==============================] - 0s 999us/step - loss: 3.3301 - accuracy: 0.0645
Epoch 3/100
1/1 [==============================] - 0s 999us/step - loss: 3.3274 - accuracy: 0.0968
Epoch 4/100
1/1 [==============================] - 0s 999us/step - loss: 3.3244 - accuracy: 0.0968
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 3.3209 - accuracy: 0.0968
Epoch 6/100
1/1 [==============================] - 0s 999us/step - loss: 3.3166 - accuracy: 0.0968
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 3.3111 - accuracy: 0.0968
Epoch 8/100
1/1 [==============================] - 0s 998us/step - loss: 3.3037 - accuracy: 0.0968
Epoch 9/100
1/1 [==============================] - 0s 998us/step - loss: 3.2932 - accuracy: 0.0968
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 3.2781 - accuracy: 0.0968
Epoch 11/100
1/

1/1 [==============================] - 0s 998us/step - loss: 1.5973 - accuracy: 0.6774
Epoch 84/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6200 - accuracy: 0.4839
Epoch 85/100
1/1 [==============================] - 0s 2ms/step - loss: 1.5638 - accuracy: 0.6129
Epoch 86/100
1/1 [==============================] - 0s 998us/step - loss: 1.5748 - accuracy: 0.5484
Epoch 87/100
1/1 [==============================] - 0s 998us/step - loss: 1.5360 - accuracy: 0.5806
Epoch 88/100
1/1 [==============================] - 0s 999us/step - loss: 1.5207 - accuracy: 0.6129
Epoch 89/100
1/1 [==============================] - 0s 3ms/step - loss: 1.5393 - accuracy: 0.4839
Epoch 90/100
1/1 [==============================] - 0s 3ms/step - loss: 1.5036 - accuracy: 0.6129
Epoch 91/100
1/1 [==============================] - 0s 999us/step - loss: 1.4731 - accuracy: 0.6129
Epoch 92/100
1/1 [==============================] - 0s 999us/step - loss: 1.4515 - accuracy: 0.6129
Epoch 93/100
1/1 [===

In [6]:
# Функция для генерации текста
def generate_text(seed_text, next_words, model, max_sequence_length, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        
    return seed_text

In [7]:
# Генерация текста
seed_text = "Утренний позитив добавляет годы"
generated_text = generate_text(seed_text, 10, model, max_sequence_length, tokenizer)
print(generated_text)

Утренний позитив добавляет годы ответ ответ ответ ответ ответ ответ проблемы проблемы проблемы думайте
